In [1]:
from tool import *
# from openpyxl import Workbook
# from openpyxl import load_workbook
# from openpyxl.utils import get_column_letter
# from openpyxl.drawing.image import Image
# from openpyxl.styles import PatternFill, Alignment, Font, Border, Side

In [2]:
df = pd.read_excel(get_filename())

In [4]:
df.insert(6, 'Inch', df['Mapping Model.Suffix'].apply(lambda x:x[:2]))

In [6]:
sp1 = df

In [9]:
c1 = sp1['From Site'].str.contains('AUO')
c2 = sp1['From Site'].str.contains('TPV')
c3 = sp1['From Site'].str.contains('TCL')
c4 = (sp1['Inch'] == '22')
c5 = (sp1['Inch'] == '24')
c6 = (sp1['Inch'] == '25')
c7 = (sp1['Inch'] == '27')
c8 = (sp1['Inch'] == '32')

In [11]:
sp1

,Year,Month,From Site,Region,Country,Subsidiary,Inch,To Site,Series,Mapping Model.Suffix,...,Unit Price,Method,SO No.,Ship Date,Week Name,Ship,BL No,Final Destination,통관전,통관후
0,2023,11,OS_TCL_CN_P,MEA,Turkey,LGETK,27,EMTK,27,27MR400-B.APDQ,...,57.98,SEA,380169456,2023-11-05,2023-10-30(W44),512,PLISZ4E12396,TRTUZ,NaN,NaN
1,2023,11,OS_TCL_CN_P,MEA,Turkey,LGETK,22,EMTK,22,22MR410-B.APDQ,...,44.55,SEA,380169456,2023-11-05,2023-10-30(W44),840,PLISZ4E12396,TRTUZ,NaN,NaN
2,2023,11,OS_TCL_CN_P,MEA,Turkey,LGETK,22,EMTK,22,22MR410-B.APDQ,...,44.55,SEA,380169460,2023-11-05,2023-10-30(W44),840,PLISZ4E12398,TRTUZ,NaN,NaN
3,2023,11,OS_TCL_CN_P,MEA,Turkey,LGETK,24,EMTK,24,24MR400-B.APDQ,...,49.13,SEA,380170323,2023-11-17,2023-11-13(W46),800,PLISZ4E13005,TRTUZ,NaN,NaN
4,2023,11,OS_AUO_CN_P,MEA,Turkey,LGETK,27,EMTK,27,27GQ50F-B.APDQ,...,97.20,SEA,380171268,2023-12-03,2023-11-27(W48),1080,PLISZ4E13381,TRTUZ,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,2023,12,OS_AUO_CN_P,EU,Italy,LGEIS,27,EEIS,27,27GQ50F-B.AEUQ,...,97.35,SEA,380171588,2023-12-15,2023-12-11(W50),429,PLISZ4E13894,ITCNG,NaN,NaN
75,2023,12,OS_AUO_CN_P,EU,Spain,LGEES,27,EEES,27,27GQ50F-B.AEUQ,...,97.35,SEA,380171580,2023-12-15,2023-12-11(W50),1287,PLISZ4E13401,ESGU2,NaN,NaN
76,2023,12,OS_AUO_CN_P,EU,UK,LGEUK,27,EEUK,27,27GQ50F-B.AEKQ,...,98.68,SEA,380171592,2023-12-16,2023-12-11(W50),429,PLISZ4E13482,GBLTT,NaN,NaN
77,2023,12,OS_TCL_CN_P,EU,Spain,LGEES,27,EEES,27,27MR400-B.AEUQ,...,58.25,SEA,380171578,2023-12-17,2023-12-11(W50),512,PLISZ4E14189,ESGU2,NaN,NaN


In [12]:
# Container 개수 계산(40FT 기준)
sp1.loc[c1 & c5 , 'Container Count'] = sp1.loc[c1 & c5, 'Ship'] / 572 / 2 # AUO 24인치 Full Stuffing 기준 572대로 개수 산출
sp1.loc[c1 & c6, 'Container Count'] = sp1.loc[c1 & c6, 'Ship'] / 416 / 2 # AUO 25인치 Full Stuffing 기준 416대로 개수 산출
sp1.loc[c1 & c7, 'Container Count'] = sp1.loc[c1 & c7, 'Ship'] / 429 / 2 # AUO 27인치 Full Stuffing 기준 429대로 개수 산출
sp1.loc[c2 & c5, 'Container Count'] = sp1.loc[c2 & c5, 'Ship'] / 800 / 2 # TPV 24인치 Full Stuffing 기준 840대로 개수 산출
sp1.loc[c2 & c8, 'Container Count'] = sp1.loc[c2 & c8, 'Ship'] / 224 / 2 # TPV 32인치 Full Stuffing 기준 800대로 개수 산출
sp1.loc[c3 & c4, 'Container Count'] = sp1.loc[c3 & c4, 'Ship'] / 840 / 2 # TCL 22인치 Full Stuffing 기준 840대로 개수 산출
sp1.loc[c3 & c5, 'Container Count'] = sp1.loc[c3 & c5, 'Ship'] / 800 / 2 # TCL 24인치 Full Stuffing 기준 800대로 개수 산출
sp1.loc[c3 & c7, 'Container Count'] = sp1.loc[c3 & c7, 'Ship'] / 512 / 2 # TCL 27인치 Full Stuffing 기준 512대로 개수 산출

In [14]:
sp1.to_clipboard()

In [2]:
with open('D:/Data/Shipment Plan_Adjusted_new.DB', 'rb') as f:
    spdb = pickle.load(f)
with open('D:/Data/sitemap.db', 'rb') as f:
    site = pickle.load(f)

In [3]:
# Monitor 전업체
mnt_spdb = spdb.groupby('Division').get_group('MNT')
mnt_spdb = mnt_spdb[~mnt_spdb['From Site'].str.contains('PEGA')]
mnt_spdb.groupby(['Planweek', 'Category', 'Created_at']).sum('SP').reset_index().iloc[-16:, ]

,Planweek,Category,Created_at,SP
85,2023-12-11(W50),Latest26w,202312121117,771096.0
86,2023-12-11(W50),Latest26w,202312141259,772568.0
87,2023-12-18(W51),Final26w,202312200913,833882.0
88,2023-12-18(W51),Final52w,202312191411,1136877.0
89,2023-12-18(W51),Latest26w,202312180928,826160.0
90,2023-12-18(W51),Latest26w,202312181509,833637.0
91,2023-12-18(W51),Latest26w,202312190851,834560.0
92,2023-12-18(W51),Latest26w,202312211119,832402.0
93,2023-12-25(W52),Final26w,202312271206,857444.0
94,2023-12-25(W52),Final52w,202312261416,1142553.0


In [4]:
i = 99

In [5]:
sp1 = mnt_spdb.merge(mnt_spdb.groupby(['Planweek', 'Category', 'Created_at']).sum('SP').reset_index().iloc[[i], :3])

In [6]:
sp1.insert(6, 'Inch', sp1['Mapping Model.Suffix'].apply(lambda x:x[:2]))
sp1 = pd.merge(sp1, site[['To Site', 'Subsidiary', 'Country']], how='left')

In [7]:
c1 = sp1['From Site'].str.contains('AUO')
c2 = sp1['From Site'].str.contains('TPV')
c3 = sp1['From Site'].str.contains('TCL')
c4 = (sp1['Inch'] == '22')
c5 = (sp1['Inch'] == '24')
c6 = (sp1['Inch'] == '25')
c7 = (sp1['Inch'] == '27')
c8 = (sp1['Inch'] == '32')

In [8]:
# Container 개수 계산(40FT 기준)
sp1.loc[c1 & c5 , 'Container Count'] = sp1.loc[c1 & c5, 'SP'] / 572 / 2 # AUO 24인치 Full Stuffing 기준 572대로 개수 산출
sp1.loc[c1 & c6, 'Container Count'] = sp1.loc[c1 & c6, 'SP'] / 416 / 2 # AUO 25인치 Full Stuffing 기준 416대로 개수 산출
sp1.loc[c1 & c7, 'Container Count'] = sp1.loc[c1 & c7, 'SP'] / 429 / 2 # AUO 27인치 Full Stuffing 기준 429대로 개수 산출
sp1.loc[c2 & c5, 'Container Count'] = sp1.loc[c2 & c5, 'SP'] / 800 / 2 # TPV 24인치 Full Stuffing 기준 840대로 개수 산출
sp1.loc[c2 & c8, 'Container Count'] = sp1.loc[c2 & c8, 'SP'] / 224 / 2 # TPV 32인치 Full Stuffing 기준 800대로 개수 산출
sp1.loc[c3 & c4, 'Container Count'] = sp1.loc[c3 & c4, 'SP'] / 840 / 2 # TCL 22인치 Full Stuffing 기준 840대로 개수 산출
sp1.loc[c3 & c5, 'Container Count'] = sp1.loc[c3 & c5, 'SP'] / 800 / 2 # TCL 24인치 Full Stuffing 기준 800대로 개수 산출
sp1.loc[c3 & c7, 'Container Count'] = sp1.loc[c3 & c7, 'SP'] / 512 / 2 # TCL 27인치 Full Stuffing 기준 512대로 개수 산출

In [9]:
c1 = sp1['Week Name'] >= '2024-01-01(W01)'
c2 = sp1['Week Name'] <= '2024-03-25(W13)'

In [10]:
price_term_map = {'EACM': 'CIF',
 'EAIL': 'CIP',
 'EAIN_DC': 'CIP',
 'EASL': 'CIP',
 'EASL_DO': 'CIF',
 'EATH': 'CIP',
 'EAVH_HCM': 'CIF',
 'EAVH_HN': 'CIF',
 'ECHK': 'CIP',
 'EEBN': 'CIP',
 'EEES': 'CIP',
 'EEDG': 'CIP',
 'EEFS': 'CIP',
 'EEHS': 'CIP',
 'EEIS': 'CIP',
 'EELA': 'CIP',
 'EEMK': 'CIP',
 'EEPL': 'CIP',
 'EEPL_CZ': 'CIP',
 'EEPT': 'CIP',
 'EERO': 'CIP',
 'EEUK': 'CIP',
 'EHAP_LGM': 'CIP',
 'EHAP_LGS': 'CIP',
 'EHAP_NZA': 'CIP',
 'EJJP': 'CIP',
 'ESPR': 'CFR',
 'EAML': 'CIP',
 'EATT': 'CIP',
 'EKHQ': 'FOB',
 'EMGF': 'CIP',
 'EMGF_ECHODUBAI_DO': 'CIP',
 'EMGF_MES_DO': 'CIP',
 'EMGF_NAAM_DO': 'CIP',
 'EMGF_PCI_DO': 'CIP',
 'EMSJ': 'CIP',
 'EMTK': 'CIF',
 'EMYK_HYE_DO': 'CIF',
 'ENCI': 'CIP',
 'ENMS': 'CIP',
 'ENUS_V': 'CIP',
 'ESCB': 'CFR',
 'ESCL_STG': 'CFR',
 'EMGF_ALTA_DO': 'CIP'}

In [11]:
sp1 = sp1[c1 & c2]

In [12]:
sp1['Price Term'] = sp1['To Site'].map(price_term_map)

In [13]:
sp1

,Planweek,Created_at,Division,From Site,Region,To Site,Inch,Mapping Model.Suffix,Rep PMS,Category,Frozen,Week Name,SP,Subsidiary,Country,Container Count,Price Term
0,2024-01-01(W01),202401021404,MNT,OS_AUO_CN_P,INDIA,EAIL,24,24GQ50F-B.ATRQ,24GQ50F-BD.CTRVMV,Final52w,N,2024-02-05(W06),572.0,LGEIL,India,0.500000,CIP
2,2024-01-01(W01),202401021404,MNT,OS_AUO_CN_P,INDIA,EAIL,27,27GQ50F-B.ATRQ,27GQ50F-BD.CTRVMV,Final52w,N,2024-03-11(W11),429.0,LGEIL,India,0.500000,CIP
3,2024-01-01(W01),202401021404,MNT,OS_AUO_CN_P,SEA,EAIN_DC,24,24GQ50F-B.ATIQ,24GQ50F-BD.CTIVMV,Final52w,N,2024-02-19(W08),312.0,LGEIN,Indonesia,0.272727,CIP
4,2024-01-01(W01),202401021404,MNT,OS_AUO_CN_P,SEA,EAIN_DC,24,24GQ50F-B.ATIQ,24GQ50F-BD.CTIVMV,Final52w,N,2024-03-25(W13),312.0,LGEIN,Indonesia,0.272727,CIP
11,2024-01-01(W01),202401021404,MNT,OS_AUO_CN_P,SEA,EAIN_DC,27,27GQ50F-B.ATIQ,27GQ50F-BD.CTIVMV,Final52w,N,2024-02-19(W08),195.0,LGEIN,Indonesia,0.227273,CIP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2155,2024-01-01(W01),202401021404,MNT,OS_TPV_CN_P,SEA,EAIN_DC,32,32MR50C-B.ATIQ,32MR50C-BB.CTIVMR,Final52w,Y,2024-01-22(W04),320.0,LGEIN,Indonesia,0.714286,CIP
2157,2024-01-01(W01),202401021404,MNT,OS_TPV_CN_P,SEA,EATH,24,24MR400-B.ATMQ,24MR400-BB.CTMRMR,Final52w,Y,2024-01-08(W02),1920.0,LGETH,Thailand,1.200000,CIP
2158,2024-01-01(W01),202401021404,MNT,OS_TPV_CN_P,SEA,EATH,32,32MR50C-B.ATMQ,32MR50C-BB.CTMVMR,Final52w,Y,2024-01-15(W03),224.0,LGETH,Thailand,0.500000,CIP
2161,2024-01-01(W01),202401021404,MNT,OS_TPV_CN_P,SEA,EAVH_HCM,24,24MR400-B.ATVQ,24MR400-BB.CTVRMR,Final52w,Y,2024-01-29(W05),800.0,LGEVH,Vietnam,0.500000,CIF


In [16]:
sp1[sp1['Price Term'].isnull()]

,Planweek,Created_at,Division,From Site,Region,To Site,Inch,Mapping Model.Suffix,Rep PMS,Category,Frozen,Week Name,SP,Subsidiary,Country,Container Count,Price Term
889,2024-01-01(W01),202401021404,MNT,OS_TCL_CN_P,MEA,EMGF_ALTA_DO,24,24MR400-B.AMAQ,24MR400-BA.CMARMV,Final52w,N,2024-02-19(W08),480.0,LGEGF,UAE,0.300,NaN
890,2024-01-01(W01),202401021404,MNT,OS_TCL_CN_P,MEA,EMGF_ALTA_DO,27,27MR400-B.AMAQ,27MR400-BB.CMARMV,Final52w,N,2024-02-19(W08),128.0,LGEGF,UAE,0.125,NaN


In [17]:
result = sp1.fillna('TBD').pivot_table('Container Count', index=['From Site', 'To Site', 'Country', 'Region', 'Price Term'], columns='Week Name', aggfunc=sum).fillna(0).reset_index()
result.head(3)

Week Name,From Site,To Site,Country,Region,Price Term,2024-01-01(W01),2024-01-08(W02),2024-01-15(W03),2024-01-22(W04),2024-01-29(W05),2024-02-05(W06),2024-02-19(W08),2024-02-26(W09),2024-03-04(W10),2024-03-11(W11),2024-03-18(W12),2024-03-25(W13)
0,OS_AUO_CN_P,EAIL,India,INDIA,CIP,0.0,0.5,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.5,0.0,0.000000
1,OS_AUO_CN_P,EAIN_DC,Indonesia,SEA,CIP,0.0,0.5,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.272727
2,OS_AUO_CN_P,EATH,Thailand,SEA,CIP,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.5,0.5,0.0,0.0,0.000000


In [18]:
result.insert(1, 'Comp', 'BS')

In [19]:
result.insert(2, 'Div', 'IT(GMZ)')

In [20]:
result.insert(3, 'POL', 'CNYTN')

In [21]:
result.loc[result['From Site'].isin(['OS_AUO_CN_P', 'OS_TPV_CN_P']), 'POL'] = 'CNXMN'

In [22]:
result.insert(8, 'DIO Y/N', 'N')

In [23]:
result.loc[result['To Site'].str.contains('_DO'), 'DIO Y/N'] = 'Y'

In [24]:
updated_time = datetime.datetime.now().strftime('%Y%m%d')
result.to_excel(f'D:/Downloads/{updated_time}_Monitor ODM container qty.xlsx')

In [51]:
# result.to_clipboard()
sp1.to_clipboard()

In [11]:
df = pd.read_clipboard()

In [40]:
df['Region'] = df['Region'].fillna('NA')

In [44]:
df.pivot_table('Container Count', index=['Region', '업체', 'FDEST', 'Country'], columns='Month', aggfunc=sum)[['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']]

Month                                Jan        Feb        Mar        Apr  \
Region 업체  FDEST Country                                                    
EU     TCL ESGU2 Spain          1.700000   0.000000   0.129492   0.644358   
           FRSRV France         0.000000   0.000000   0.534412   0.188223   
           GBLTT UK             1.700000   0.833240   1.543155   1.435731   
           HUSZ2 Hungary        0.850000   0.202381   0.847945   0.669306   
           ITCNG Italy          0.000000   1.700000   0.489746   0.771242   
           NLTLB Germany        0.000000   0.000000   1.660156   0.584717   
                 Netherlands    0.000000   0.250684   0.664062   0.322179   
           PLKY2 Czech          0.000000   0.332031   0.332031   0.233887   
           PLKY4 Poland         0.850000   1.700000   0.000000   0.898125   
           PTAZA Portugal       0.850000   0.000000   0.000000   0.299375   
           ROGRG Romania        0.000000   0.000000   0.000000   0.000000   
INDIA  TCL INNSA India          3.400000   3.400000   4.574973   4.083324   
KOREA  TCL KRPUS Korea         14.450000  10.200000   5.100000   6.928082   
MEA    TCL AEJEA UAE           22.071667  19.535833  19.529888  20.059606   
           ILASH Israel         1.700000   2.550000   0.000000   1.394455   
           SARUH Saudi Arabia   1.700000   3.400000   1.530730   2.175589   
           TRTUZ Turkey         1.700000   1.700000   2.091164   1.801690   
NA     TCL CANYK Canada         5.090672   5.975804   8.535859   6.873589   
           USTXW US             0.000000   5.810484   8.789911   5.119651   
SCA    TCL CLVAP Chile          1.020000   2.220372   1.214286   1.647981   
           COBUN Columbia       1.700000   2.550000   1.535060   2.140158   
           MXMEX Mexico         0.850000   1.700000   1.619364   1.542438   
SEA    TCL AUBNE Austrailia     0.000000   0.161721   0.513877   0.222924   
           AUFRE Austrailia     0.000000   0.147359   0.295511   0.146132   
           AUMEL Austrailia     1.847688   2.097723   1.112140   1.668818   
           AUSYD Austrailia     1.678750   0.558372   2.698620   1.628625   
           HKHKG HongKong       0.514648   0.968709   0.739367   0.733423   
           IDCAK Indonesia      2.381771   3.161696   0.384524   1.956033   
           JPKWS Japan          0.956250   0.205417   1.066548   0.632642   
           MYPKG Malaysia       1.367969   0.927031   0.423685   0.897072   
           NZAKL New Zealand    1.436500   1.009375   2.521313   1.639001   
           PHMNL Philippines    0.000000  10.710000   0.513833   3.703478   
           SGSIN Singapore      2.083828   1.813289   0.457406   1.436843   
           THRYG Thailand       3.725470   3.142281   1.286574   2.690646   
           VNHPH Vietnam        0.670071   1.000837   0.251743   0.634409   
           VNSGN Vietnam        0.985500   1.153413   0.552800   0.888173   

Month                                May        Jun        Jul        Aug  \
Region 업체  FDEST Country                                                    
EU     TCL ESGU2 Spain          0.644358   0.644358   0.675692   0.675692   
           FRSRV France         0.188223   0.188223   0.197376   0.197376   
           GBLTT UK             1.435731   1.435731   1.505548   1.505548   
           HUSZ2 Hungary        0.669306   0.669306   0.701853   0.701853   
           ITCNG Italy          0.771242   0.771242   0.808746   0.808746   
           NLTLB Germany        0.584717   0.584717   0.613151   0.613151   
                 Netherlands    0.322179   0.322179   0.337846   0.337846   
           PLKY2 Czech          0.233887   0.233887   0.245260   0.245260   
           PLKY4 Poland         0.898125   0.898125   0.941800   0.941800   
           PTAZA Portugal       0.299375   0.299375   0.313933   0.313933   
           ROGRG Romania        0.000000   0.000000   0.000000   0.000000   
INDIA  TCL INNSA India          4.083324   4.083324   4.229157   4.22915

## Container 개수 계산

In [2]:
ver_table = spdb.groupby(['Planweek', 'From Site', 'Category', 'Created_at']).sum('SP').reset_index()
ver_table = ver_table[ver_table['From Site'].str.contains('TCL')]
ver_table.iloc[-30:, ]

,Planweek,From Site,Category,Created_at,SP
291,2023-10-30(W44),OS_TCL_CN_P,Latest26w,202310301019,294107.0
292,2023-10-30(W44),OS_TCL_CN_P,Latest26w,202310301517,362751.0
293,2023-10-30(W44),OS_TCL_CN_P,Latest26w,202310310857,384275.0
322,2023-11-06(W45),OS_TCL_CN_P,Final26w,202311080925,395040.0
323,2023-11-06(W45),OS_TCL_CN_P,Final52w,202311071508,526779.0
324,2023-11-06(W45),OS_TCL_CN_P,Latest26w,202311061125,386473.0
325,2023-11-06(W45),OS_TCL_CN_P,Latest26w,202311061536,393395.0
326,2023-11-06(W45),OS_TCL_CN_P,Latest26w,202311061759,394746.0
327,2023-11-06(W45),OS_TCL_CN_P,Latest26w,202311070846,397496.0
361,2023-11-13(W46),OS_TCL_CN_P,Final26w,202311150835,406326.0


In [3]:
i1 = 430

In [4]:
spdb.groupby(['Planweek', 'Category', 'Created_at']).sum('SP').reset_index().iloc[-16:,]

,Planweek,Category,Created_at,SP
62,2023-11-13(W46),Latest26w,202311151420,1002887.0
63,2023-11-13(W46),Latest26w,202311181107,1003309.0
64,2023-11-20(W47),Final26w,202311220928,1063448.0
65,2023-11-20(W47),Final52w,202311211424,1473462.0
66,2023-11-20(W47),Latest26w,202311200946,1062242.0
67,2023-11-20(W47),Latest26w,202311201503,1065869.0
68,2023-11-20(W47),Latest26w,202311210904,1072286.0
69,2023-11-20(W47),Latest26w,202311230835,1063960.0
70,2023-11-27(W48),Final26w,202311290837,1096714.0
71,2023-11-27(W48),Final52w,202311281435,1475801.0


In [4]:
sp1 = spdb.merge(spdb.groupby(['Planweek', 'From Site', 'Category', 'Created_at']).sum('SP').reset_index().iloc[[i1], :4])

In [5]:
sp1.insert(6, 'Inch', sp1['Mapping Model.Suffix'].apply(lambda x:x[:2]))
sp1 = pd.merge(sp1, site[['To Site', 'Subsidiary', 'Country']], how='left')

In [6]:
sp1.loc[sp1['Inch'] == '22', 'Container Count'] = sp1.loc[sp1['Inch'] == '22', 'SP'] / 840 # 22인치 Full Stuffing 기준 840대로 개수 산출
sp1.loc[sp1['Inch'] == '24', 'Container Count'] = sp1.loc[sp1['Inch'] == '24', 'SP'] / 800 # 24인치 Full Stuffing 기준 800대로 개수 산출
sp1.loc[sp1['Inch'] == '27', 'Container Count'] = sp1.loc[sp1['Inch'] == '27', 'SP'] / 512 # 27인치 Full Stuffing 기준 512대로 개수 산출

In [7]:
c1 = sp1['Week Name'] >= '2023-11-27(W48)'
c2 = sp1['Week Name'] <= '2024-02-26(W09)'

In [8]:
price_term_map = {'EACM': 'CIF',
 'EAIL': 'CIP',
 'EAIN_DC': 'CIP',
 'EASL': 'CIP',
 'EASL_DO': 'CIF',
 'EATH': 'CIP',
 'EAVH_HCM': 'CIF',
 'EAVH_HN': 'CIF',
 'ECHK': 'CIP',
 'EEBN': 'CIP',
 'EEES': 'CIP',
 'EEFS': 'CIP',
 'EEHS': 'CIP',
 'EEIS': 'CIP',
 'EELA': 'CIP',
 'EEMK': 'CIP',
 'EEPL': 'CIP',
 'EEPL_CZ': 'CIP',
 'EEPT': 'CIP',
 'EERO': 'CIP',
 'EEUK': 'CIP',
 'EHAP_LGM': 'CIP',
 'EHAP_LGS': 'CIP',
 'EHAP_NZA': 'CIP',
 'EJJP': 'CIP',
 'EKHQ': 'FOB',
 'EMGF': 'CIP',
 'EMGF_ECHODUBAI_DO': 'CIP',
 'EMGF_MES_DO': 'CIP',
 'EMGF_NAAM_DO': 'CIP',
 'EMGF_PCI_DO': 'CIP',
 'EMSJ': 'CIP',
 'EMTK': 'CIF',
 'EMYK_HYE_DO': 'CIF',
 'ENCI': 'CIP',
 'ENMS': 'CIP',
 'ENUS_V': 'CIP',
 'ESCB': 'CFR',
 'ESCL_STG': 'CFR'}

In [9]:
sp1 = sp1[c1 & c2]

In [10]:
sp1['Price Term'] = sp1['To Site'].map(price_term_map)

In [11]:
result = sp1.pivot_table('Container Count', index=['From Site', 'To Site', 'Country', 'Region', 'Price Term'], columns='Week Name', aggfunc=sum).fillna(0).reset_index()
result.head(3)

Week Name,From Site,To Site,Country,Region,Price Term,2023-11-27(W48),2023-12-04(W49),2023-12-11(W50),2023-12-18(W51),2023-12-25(W52),2024-01-01(W01),2024-01-08(W02),2024-01-15(W03),2024-01-22(W04),2024-01-29(W05),2024-02-05(W06),2024-02-19(W08),2024-02-26(W09)
0,OS_TCL_CN_P,EACM,Philippines,SEA,CIF,0.0,1.0,0.0,2.4,3.4,0.0,0.0,0.0,0.0,4.004375,0.312500,0.000000,5.857292
1,OS_TCL_CN_P,EAIL,India,INDIA,CIP,2.0,3.0,4.0,3.0,2.0,1.0,2.0,4.0,3.0,4.000000,3.000000,2.000000,2.000000
2,OS_TCL_CN_P,EAIN_DC,Indonesia,SEA,CIP,0.0,0.0,0.0,0.0,0.0,3.6,2.6,0.0,0.0,0.933333,2.306548,0.969048,0.595238


In [12]:
result.insert(1, 'Comp', 'BS')

In [13]:
result.insert(2, 'Div', 'IT(GMZ)')

In [14]:
result.insert(3, 'POL', 'CNYTN')

In [15]:
result.to_clipboard()

In [11]:
df = pd.read_clipboard()

In [40]:
df['Region'] = df['Region'].fillna('NA')

In [44]:
df.pivot_table('Container Count', index=['Region', '업체', 'FDEST', 'Country'], columns='Month', aggfunc=sum)[['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']]

Month                                Jan        Feb        Mar        Apr  \
Region 업체  FDEST Country                                                    
EU     TCL ESGU2 Spain          1.700000   0.000000   0.129492   0.644358   
           FRSRV France         0.000000   0.000000   0.534412   0.188223   
           GBLTT UK             1.700000   0.833240   1.543155   1.435731   
           HUSZ2 Hungary        0.850000   0.202381   0.847945   0.669306   
           ITCNG Italy          0.000000   1.700000   0.489746   0.771242   
           NLTLB Germany        0.000000   0.000000   1.660156   0.584717   
                 Netherlands    0.000000   0.250684   0.664062   0.322179   
           PLKY2 Czech          0.000000   0.332031   0.332031   0.233887   
           PLKY4 Poland         0.850000   1.700000   0.000000   0.898125   
           PTAZA Portugal       0.850000   0.000000   0.000000   0.299375   
           ROGRG Romania        0.000000   0.000000   0.000000   0.000000   
INDIA  TCL INNSA India          3.400000   3.400000   4.574973   4.083324   
KOREA  TCL KRPUS Korea         14.450000  10.200000   5.100000   6.928082   
MEA    TCL AEJEA UAE           22.071667  19.535833  19.529888  20.059606   
           ILASH Israel         1.700000   2.550000   0.000000   1.394455   
           SARUH Saudi Arabia   1.700000   3.400000   1.530730   2.175589   
           TRTUZ Turkey         1.700000   1.700000   2.091164   1.801690   
NA     TCL CANYK Canada         5.090672   5.975804   8.535859   6.873589   
           USTXW US             0.000000   5.810484   8.789911   5.119651   
SCA    TCL CLVAP Chile          1.020000   2.220372   1.214286   1.647981   
           COBUN Columbia       1.700000   2.550000   1.535060   2.140158   
           MXMEX Mexico         0.850000   1.700000   1.619364   1.542438   
SEA    TCL AUBNE Austrailia     0.000000   0.161721   0.513877   0.222924   
           AUFRE Austrailia     0.000000   0.147359   0.295511   0.146132   
           AUMEL Austrailia     1.847688   2.097723   1.112140   1.668818   
           AUSYD Austrailia     1.678750   0.558372   2.698620   1.628625   
           HKHKG HongKong       0.514648   0.968709   0.739367   0.733423   
           IDCAK Indonesia      2.381771   3.161696   0.384524   1.956033   
           JPKWS Japan          0.956250   0.205417   1.066548   0.632642   
           MYPKG Malaysia       1.367969   0.927031   0.423685   0.897072   
           NZAKL New Zealand    1.436500   1.009375   2.521313   1.639001   
           PHMNL Philippines    0.000000  10.710000   0.513833   3.703478   
           SGSIN Singapore      2.083828   1.813289   0.457406   1.436843   
           THRYG Thailand       3.725470   3.142281   1.286574   2.690646   
           VNHPH Vietnam        0.670071   1.000837   0.251743   0.634409   
           VNSGN Vietnam        0.985500   1.153413   0.552800   0.888173   

Month                                May        Jun        Jul        Aug  \
Region 업체  FDEST Country                                                    
EU     TCL ESGU2 Spain          0.644358   0.644358   0.675692   0.675692   
           FRSRV France         0.188223   0.188223   0.197376   0.197376   
           GBLTT UK             1.435731   1.435731   1.505548   1.505548   
           HUSZ2 Hungary        0.669306   0.669306   0.701853   0.701853   
           ITCNG Italy          0.771242   0.771242   0.808746   0.808746   
           NLTLB Germany        0.584717   0.584717   0.613151   0.613151   
                 Netherlands    0.322179   0.322179   0.337846   0.337846   
           PLKY2 Czech          0.233887   0.233887   0.245260   0.245260   
           PLKY4 Poland         0.898125   0.898125   0.941800   0.941800   
           PTAZA Portugal       0.299375   0.299375   0.313933   0.313933   
           ROGRG Romania        0.000000   0.000000   0.000000   0.000000   
INDIA  TCL INNSA India          4.083324   4.083324   4.229157   4.22915

In [14]:
df.loc[df['Series']==22, 'Jan_Container']

,업체,Region,FDEST,To Site,Country,Series,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,TCL,EU,ESGU2,EEES,Spain,22,714.0,0.00,0.00,251.475133,251.475133,251.475133,263.703938,263.703938,263.703938,321.341921,321.341921,321.341921
2,TCL,EU,FRSRV,EEFS,France,22,0.0,0.00,170.00,59.875032,59.875032,59.875032,62.786652,62.786652,62.786652,76.509981,76.509981,76.509981
4,TCL,EU,GBLTT,EEUK,UK,22,1428.0,527.00,850.00,987.938021,987.938021,987.938021,1035.979755,1035.979755,1035.979755,1262.414690,1262.414690,1262.414690
6,TCL,EU,HUSZ2,EEMK,Hungary,22,0.0,170.00,425.00,209.562611,209.562611,209.562611,219.753281,219.753281,219.753281,267.784934,267.784934,267.784934
8,TCL,EU,ITCNG,EEIS,Italy,22,0.0,714.00,0.00,251.475133,251.475133,251.475133,263.703938,263.703938,263.703938,321.341921,321.341921,321.341921
15,TCL,EU,ROGRG,EERO,Romania,22,0.0,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
17,TCL,INDIA,INNSA,EAIL,India,22,1428.0,2142.00,2162.40,2057.784615,2057.784615,2057.784615,2131.276923,2131.276923,2131.276923,2175.372308,2175.372308,2175.372308
20,TCL,KOREA,KRPUS,EKHQ,Korea,22,1428.0,1428.00,714.00,831.369863,831.369863,831.369863,831.369863,831.369863,831.369863,774.315069,774.315069,774.315069
22,TCL,MEA,AEJEA,EMGF,UAE,22,714.0,714.00,144.50,515.948293,515.948293,515.948293,582.101384,582.101384,582.101384,512.426133,512.426133,512.426133
27,TCL,MEA,SARUH,EMSJ,Saudi Arabia,22,0.0,714.00,0.00,234.268414,234.268414,234.268414,264.305493,264.305493,264.305493,232.669163,232.669163,232.669163


In [107]:
with open('D:/Data/Shipment Plan_Adjusted_new.DB', 'rb') as f:
    spdb = pickle.load(f)

# with open('D:/Data/contact_window.db', 'rb') as f:
#     pic = pickle.load(f)

# with open('D:/Data/ODM PO.db', 'rb') as f:
#     po = pickle.load(f)
    
# with open('D:/Data/TCL PP.bin', 'rb') as f:
#     pp_db = pickle.load(f)

with open(f'D:/Data/ODM_SR.db', 'rb') as f:
    sr = pickle.load(f)

with open('D:/Data/sitemap.db', 'rb') as f:
    ste = pickle.load(f)

In [93]:
ver_table = spdb.groupby(['Planweek', 'Category', 'Created_at']).sum('SP').reset_index()
# ver_table = ver_table[ver_table['From Site'].str.contains('TPV')]
ver_table.iloc[-30:, ]

,Planweek,Category,Created_at,SP
8,2023-08-21(W34),Final52w,202309081354,1029172.0
9,2023-08-28(W35),Final26w,202309081213,785053.0
10,2023-08-28(W35),Final52w,202309081355,1075029.0
11,2023-09-04(W36),Final26w,202309070918,829926.0
12,2023-09-04(W36),Final52w,202309051516,1106535.0
13,2023-09-11(W37),Final26w,202309130928,851450.0
14,2023-09-11(W37),Final52w,202309121428,1204791.0
15,2023-09-11(W37),Latest26w,202309111606,846961.0
16,2023-09-11(W37),Latest26w,202309120943,846614.0
17,2023-09-18(W38),Final26w,202309201227,948121.0


## GSCP 특정 시점 SP기준으로 계산

In [108]:
i = 34
sp = pd.merge(spdb, spdb.groupby(['Planweek', 'Category', 'Created_at']).sum('SP').reset_index().iloc[[i], :3])
# sp = sp[['To Site', 'Mapping Model.Suffix', 'Week Name', 'Category', 'SP']]
# sp = sp.rename(columns={'SP':'QTY'})
# sp['Category'] = '2_SP'

In [109]:
sp = sp[sp['From Site'].isin(['OS_AUO_CN_P', 'OS_TPV_CN_P', 'OS_TCL_CN_P'])] # 모니터 ODM 업체만 추출
c1 = sp['From Site'].str.contains('AUO|TPV|TCL') # 모니터 ODM 일 경우
c2 = sp['Mapping Model.Suffix'].str.match('\d{2}[T|Z|U|A][G|D|N|W|C]?\d{2}[A-Z]{1,2}') # PC/태블릿 모델일 경우
c3 = sp['Mapping Model.Suffix'].str.contains('CL600') # 페가트론 CL600 모델일 경우

sp.loc[c1, 'Series'] = sp[c1]['Mapping Model.Suffix'].apply(lambda x:x[:2]) # 모니터 모델의 시리즈명(inch) 추출
sp.loc[c2, 'Series'] = sp[c2]['Mapping Model.Suffix'].apply(lambda x:x.split('-')[0]).str.replace('D|G|N|W|C', '').apply(lambda x:x[:6]) # PC모델의 대표모델명 추출
sp.loc[c3, 'Series'] = 'CL600'

sp['Month'] = sp['Week Name'].apply(lambda x:f'{datetime.date.fromisoformat(x[:10]).isocalendar().year}-{get_month_from_date(datetime.date.fromisoformat(x[:10])):02d}')

sp = sp[sp['Month'] < '2024-01'] # 23년 실적만 추출
sp = sp[sp['Month'] > '2023-09'] # 23년 10월부터 실적만 추출

C:\Users\paul76.lee\AppData\Local\Temp\ipykernel_12208\8560109.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  sp.loc[c2, 'Series'] = sp[c2]['Mapping Model.Suffix'].apply(lambda x:x.split('-')[0]).str.replace('D|G|N|W|C', '').apply(lambda x:x[:6]) # PC모델의 대표모델명 추출


In [108]:

sp2024 = sp[sp['Week Name'] >= '2024-01-01(W01)'].copy() # 24년 SP만 추출
sp2024['Month'] = sp2024['Week Name'].apply(lambda x:f'{datetime.date.fromisoformat(x[:10]).isocalendar().year}-{get_month_from_date(datetime.date.fromisoformat(x[:10])):02d}')
sp2024 = pd.merge(sp2024 , ste[['To Site', 'Subsidiary', 'Country']], how='left')

C:\Users\paul76.lee\AppData\Local\Temp\ipykernel_20348\1243651506.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  sp.loc[c2, 'Series'] = sp[c2]['Mapping Model.Suffix'].apply(lambda x:x.split('-')[0]).str.replace('D|G|N|W|C', '').apply(lambda x:x[:6]) # PC모델의 대표모델명 추출


In [110]:
ship_to_map = {'LG Electronics United Kingdom Ltd.':'EEUK', 'LG ELECTRONICS DEUTSCHLAND.GMBH.':'EEDG', 'ENUS_TX':'ENUS_V', 'ENUS_CA':'ENUS_V'}
sr['To Site'] = sr['To Site'].replace(ship_to_map)
sr = sr.reset_index()
c1 = sr['From Site'].str.contains('AUO|TPV|TCL') # 모니터 ODM 일 경우
c2 = sr['Mapping Model.Suffix'].str.match('\d{2}[T|Z|U|A][G|D|N|W|C]?\d{2}[A-Z]{1,2}') # PC/태블릿 모델일 경우
c3 = sr['Mapping Model.Suffix'].str.contains('CL600') # 페가트론 CL600 모델일 경우
sr.loc[c1, 'Series'] = sr[c1]['Mapping Model.Suffix'].apply(lambda x:x[:2]) # 모니터 모델의 시리즈명(inch) 추출
sr.loc[c2, 'Series'] = sr[c2]['Mapping Model.Suffix'].apply(lambda x:x.split('-')[0]).str.replace('D|G|N|W|C', '').apply(lambda x:x[:6]) # PC모델의 대표모델명 추출
sr.loc[c3, 'Series'] = 'CL600'
sr['Week Name'] = sr['Ship Date'].apply(get_weekname)
sr['Month'] = sr['Week Name'].apply(lambda x:f'{datetime.date.fromisoformat(x[:10]).isocalendar().year}-{get_month_from_date(datetime.date.fromisoformat(x[:10])):02d}')

sr = sr[sr['From Site'].isin(['OS_AUO_CN_P', 'OS_TPV_CN_P', 'OS_TCL_CN_P'])] # 모니터 ODM 업체만 추출
sr = pd.merge(sr , ste[['To Site', 'Subsidiary', 'Country', 'Region']], how='left')

C:\Users\paul76.lee\AppData\Local\Temp\ipykernel_12208\1468969924.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  sr.loc[c2, 'Series'] = sr[c2]['Mapping Model.Suffix'].apply(lambda x:x.split('-')[0]).str.replace('D|G|N|W|C', '').apply(lambda x:x[:6]) # PC모델의 대표모델명 추출


In [111]:
sp = pd.merge(sp, ste[['To Site', 'Country']], how='left')[['From Site', 'Region', 'Country', 'To Site', 'Mapping Model.Suffix', 'Week Name', 'SP', 'Series', 'Month']]

In [112]:
sr = sr[['From Site', 'Region', 'Country', 'To Site', 'Mapping Model.Suffix', 'Week Name', 'Ship', 'Series', 'Month']]
sr = sr[sr['Month'] > '2022-12']
sr = sr.rename(columns={'Ship':'SP'})

In [113]:
sp2023 = pd.concat([sr, sp])

In [116]:
sp2023.to_clipboard()

In [115]:
sp2023.pivot_table('SP', index=['From Site', 'Region', 'Country', 'To Site', 'Mapping Model.Suffix'], columns=['Month', 'Week Name'], aggfunc=sum).to_clipboard()

In [132]:
df1 = pd.read_clipboard()

In [135]:
pd.merge(df1[['업체', 'To Site']], ste[['To Site', 'Subsidiary', 'Region', 'Country']], how='left')[['업체', 'Region']].to_clipboard(index=False)

In [130]:
ste[ste['To Site'].str.contains('SA')]

,To Site,Subsidiary,Region,Subsidiary2,Country
36,EMGF_SALEM_DO,LGEGF,MEA,LGEGF,UAE
41,EFSA,LGESA,MEA,LGESA,South Africa
42,EFSA_DRAGON_DO,LGESA,MEA,LGESA,South Africa


In [112]:
fdest_map = sr[sr['Division']=='MNT'][['To Site', 'Final Destination']].drop_duplicates().groupby('To Site')['Final Destination'].max()
fdest_map['EKHQ'] = 'KRPUS'
fdest_map

To Site
EAIL              INNSA
EAIN_DC           IDCAK
EASL              SGSIN
EATH              THRYG
EATT              TWKEL
EAVH_HCM          VNSGN
EAVH_HN           VNHPH
ECHK              HKHKG
EEBN              NLTLB
EEDG              NLTLB
EEES              ESGU2
EEFS              FRSRV
EEHS              GRASS
EEIS              ITCNG
EEMK              HUSZ2
EEPL              PLKY4
EEPL_CZ           PLKY2
EEPT              PTAZA
EERO              ROGRG
EESW              SEJKG
EESW_ELKJOP_DO    SEJKG
EEUK              GBLTT
EFEG              EGPSD
EHAP_LGM          AUMEL
EHAP_LGS          AUSYD
EHAP_NZA          NZAKL
EJJP              JPKWS
EKHQ              KRPUS
EMGF              AEJEA
EMSJ              SARUH
EMSJ_EMAM_M_DO    SARUH
EMTK              TRTUZ
EMYK_HYE_DO       ILASH
ENCI              CANYK
ENMS              MXMEX
ENUS_V            USTXW
ESCB              COBUN
Name: Final Destination, dtype: object

In [113]:
sp2024['FDEST'] = sp2024['To Site'].map(fdest_map)

In [115]:
sp2024.fillna('TBD').pivot_table('SP', index=['Region', 'FDEST', 'To Site', 'Country', 'Series'], columns='Month', aggfunc=sum).to_clipboard()

In [131]:
df = pd.read_clipboard()

In [148]:
df = pd.merge(df, ste[['Subsidiary', 'Region', 'To Site']].groupby('Subsidiary')['To Site'].max().reset_index(), how='left')

In [150]:
pd.merge(df, ste[['Subsidiary', 'Region', 'To Site']].groupby('Subsidiary')['Region'].max().reset_index(), how='left' ).to_clipboard()

In [122]:
pd.merge(df , ste[['Subsidiary', 'Region', 'Country']], how='left')

,Model 2,Subsidiary,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Region,Country
0,27GS60QC,LGEAP,500,800,500,500,500,500,500,1200,100,SEA,Austrailia
1,27GS60QC,LGEAP,500,800,500,500,500,500,500,1200,100,SEA,Austrailia
2,27GS60QC,LGEAP,500,800,500,500,500,500,500,1200,100,SEA,Austrailia
3,27GS60QC,LGEAP,500,800,500,500,500,500,500,1200,100,SEA,Austrailia
4,27GS60QC,LGEAP,500,800,500,500,500,500,500,1200,100,SEA,New Zealand
...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,32GS60QC,LGEUS,2500,2500,2500,2500,2500,2500,2500,2500,2500,NA,US
170,32GS60QC,LGEUS,2500,2500,2500,2500,2500,2500,2500,2500,2500,NA,US
171,32GS60QC,LGEVH,0,0,0,0,0,0,0,0,0,SEA,Vietnam
172,32GS60QC,LGEVH,0,0,0,0,0,0,0,0,0,SEA,Vietnam


In [ ]:
{'EACM':'PHMNL', 'EAML':'MYPKG', 'EASL_DO':'SGSIN', ''}

In [17]:
sr[['To Site', 'Final Destination']].drop_duplicates().groupby('To Site')['Final Destination'].unique()

To Site
EAIL                                                [INNSA]
EAIN_DC                                             [IDCAK]
EASL                                                [SGSIN]
EATH                                         [THRYG, THBGN]
EATT                                         [TWTPE, TWKEL]
EAVH_HCM                                            [VNSGN]
EAVH_HN                                      [VNHAN, VNHPH]
ECHK                                                [HKHKG]
EEBN                                                [NLTLB]
EEDG                                         [NLTLB, DEFRA]
EEES                                                [ESGU2]
EEFS                                                [FRSRV]
EEHS                                                [GRASS]
EEIS                                                [ITCNG]
EEMK                                                [HUSZ2]
EEPL                                                [PLKY4]
EEPL_CZ                         

In [65]:
opo = pd.merge(po.loc['OS_TCL_CN_P'].reset_index(), sr.loc['OS_TCL_CN_P'].groupby('PO No.').sum('Ship')['Ship'].reset_index(), how='left').fillna(0)
opo['Week Name'] = opo['RSD'].map(get_weekname)
opo['Ship'] = opo['Ship'].astype(int)
opo['QTY'] = opo['PO'] - opo['Ship']
opo = opo[opo['QTY'] > 0]
opo['Category'] = '3_PO'
opo = opo[['To Site', 'Mapping Model.Suffix', 'Week Name', 'Category', 'QTY']]
ship_to_map = {'LG Electronics United Kingdom Ltd.':'EEUK', 'LG ELECTRONICS DEUTSCHLAND.GMBH.':'EEDG', 'ENUS_TX':'ENUS_V', 'ENUS_CA':'ENUS_V'}
opo['To Site'] = opo['To Site'].replace(ship_to_map)
ref_wk = spdb.groupby(['Planweek', 'Created_at', 'Category', 'From Site']).sum('SP').reset_index().iloc[i]['Planweek']
opo.loc[opo['Week Name'] < ref_wk, 'Week Name'] = ref_wk

In [66]:
pp1 = pp_db[pp_db['Updated_at'] == pp_db['Updated_at'].unique()[-1]].reset_index(drop=True) # 최신 PP 를 가져옴
pp2 = pp_db[pp_db['Updated_at'] == pp_db['Updated_at'].unique()[-2]].reset_index(drop=True)# 이전 Version의 PP를 가져옴
pp_sum = pd.concat([pp1, pp2]).reset_index(drop=True)
pp_sum['Month'] = pp_sum['PP Date'].apply(lambda x:f'{x.isocalendar().year}-{get_month_from_date(x):02d}')
pp_sum['Week Name'] = pp_sum['PP Date'].map(get_weekname)
pp_sum['Series'] = pp_sum['Mapping Model.Suffix'].apply(lambda x:x.split('-')[0]).apply(lambda x:x.split('.')[0])
pp_sum['Day'] = pp_sum['PP Date'].dt.strftime('%a')
pp_sum['Date'] = pp_sum['PP Date'].dt.strftime('%m/%d')
pp_sum = pp_sum.merge(ste[['To Site', 'Region', 'Country']], how='left')

gap_month = pp_sum.groupby(['Region', 'Country', 'To Site', 'Mapping Model.Suffix', 'TCL BOM', 'Month', 'Week Name', 'Date', 'Day', 'Updated_at']).sum('QTY').unstack('Updated_at').fillna(0).astype(int)
gap_month['Gap'] = gap_month[gap_month.columns[-1]] - gap_month[gap_month.columns[0]]
gap_month = gap_month.unstack(['Month', 'Week Name']).dropna(how='all', axis=1).fillna(0).astype(int)
gap_month['Sum'] = gap_month['Gap'].sum(axis=1)
# gap_month.loc[('', '', '', '', '', 'Sum'), :] = gap_month.sum()
gap_month = gap_month.astype(int)

In [67]:
pp1['Week Name'] = pp1['PP Date'].map(get_weekname)
pp1 = pp1[pp1['Week Name'] >= sp['Week Name'].min()]
pp1['Category'] = '4_PP'
pp = pp1[['To Site', 'Mapping Model.Suffix', 'Week Name', 'Category', 'QTY']].copy()

In [68]:
p1 = pd.concat([sp, opo, pp])[['To Site', 'Mapping Model.Suffix']].drop_duplicates().reset_index(drop=True)
p1['Category'] = '1_BOH'
psi = pd.concat([p1, p1.drop('Category', axis=1)]).fillna('2_SP').reset_index(drop=True)
psi = pd.concat([psi, p1.drop('Category', axis=1)]).fillna('3_PO').reset_index(drop=True)
psi = pd.concat([psi, p1.drop('Category', axis=1)]).fillna('4_PP').reset_index(drop=True)
psi = pd.concat([psi, p1.drop('Category', axis=1)]).fillna('5_ETD').reset_index(drop=True)
psi = pd.concat([psi, p1.drop('Category', axis=1)]).fillna('6_SP Gap').reset_index(drop=True)

psi = psi.set_index(['To Site', 'Mapping Model.Suffix', 'Category'])
min_wk = pd.concat([sp, opo, pp])['Week Name'].min()
max_wk = pd.concat([sp, opo, pp])['Week Name'].max()
wk = min_wk
wklist = [get_weekname_from(wk, i) for i in range(60)]
wklist = wklist[:wklist.index(max_wk)+1]
psi[wklist] = 0
psi.update(sp.pivot_table('QTY', index=['To Site', 'Mapping Model.Suffix', 'Category'], columns='Week Name', aggfunc=sum).fillna(0).astype(int))
psi.update(opo.pivot_table('QTY', index=['To Site', 'Mapping Model.Suffix', 'Category'], columns='Week Name', aggfunc=sum).fillna(0).astype(int))
psi.update(pp.pivot_table('QTY', index=['To Site', 'Mapping Model.Suffix', 'Category'], columns='Week Name', aggfunc=sum).fillna(0).astype(int))
psi = psi.convert_dtypes().sort_index()

In [69]:
boh = pd.read_excel(get_filename(), skiprows=1)
boh = boh[boh['Category'] == '1_BOH']
boh = boh[['To Site', 'Mapping Model.Suffix', 'Category', min_wk]]
boh = boh[boh[min_wk] > 0].set_index(['To Site', 'Mapping Model.Suffix', 'Category'])
boh

,,,2023-09-04(W36)
To Site,Mapping Model.Suffix,Category,
EMTK,22MR410-B.APDQ,1_BOH,840


In [70]:
psi.update(boh)

In [71]:
for site, model in p1[['To Site', 'Mapping Model.Suffix']].values:
    for week in psi.columns:
        if week == psi.columns[0]: # 계산하는 Week가 시작하는 Week 일 경우
            current_boh = psi.loc[(site, model, '1_BOH'), week] # 금주 boh
            current_sp = psi.loc[(site, model, '2_SP'), week] # 금주 sp
            if current_boh > current_sp :
                psi.loc[(site, model, '5_ETD'), week] = current_sp
            else:
                psi.loc[(site, model, '5_ETD'), week] = current_boh
            psi.loc[(site, model, '6_SP Gap'), week] = psi.loc[(site, model, '5_ETD'), week] - current_sp
        else:
            last_boh = psi.loc[(site, model, '1_BOH'), get_weekname_from(week, -1)]
            last_pp = psi.loc[(site, model, '4_PP'), get_weekname_from(week, -1)]
            last_etd = psi.loc[(site, model, '5_ETD'), get_weekname_from(week, -1)]
            psi.loc[(site, model, '1_BOH'), week] =  last_boh + last_pp - last_etd
            current_boh = psi.loc[(site, model, '1_BOH'), week]
            current_sp = psi.loc[(site, model, '2_SP'), week]
            last_gap = psi.loc[(site, model, '6_SP Gap'), get_weekname_from(week, -1)]
            if current_boh  > current_sp - last_gap:
                psi.loc[(site, model, '5_ETD'), week] = current_sp - last_gap
            else:
                psi.loc[(site, model, '5_ETD'), week] = current_boh
            psi.loc[(site, model, '6_SP Gap'), week] = psi.loc[(site, model, '5_ETD'), week] - current_sp + last_gap
psi = pd.merge(psi.reset_index(), ste[['To Site', 'Region', 'Country']], how='left').set_index(['Region', 'Country', 'To Site', 'Mapping Model.Suffix', 'Category'])
psi = psi.reset_index()
psi.insert(4, 'TCL BOM', psi['Mapping Model.Suffix'].map(pp_db[['Mapping Model.Suffix', 'TCL BOM']].groupby('Mapping Model.Suffix')['TCL BOM'].max()))
psi = psi.set_index(['Region', 'Country', 'To Site', 'Mapping Model.Suffix', 'TCL BOM', 'Category'])

In [72]:
# pp_daily = pp_sum.pivot_table('QTY', index=['Region', 'Country', 'To Site', 'Mapping Model.Suffix', 'Date', 'Day'], columns=['Updated_at', 'Month'], aggfunc=sum).fillna('-')
update_time = datetime.datetime.now().strftime('%Y%m%d%H')
filename = f'{update_time}_TCL Monitor PSI.xlsx'
with pd.ExcelWriter(f'D:/MNT/TCL/PSI/{filename}') as writer:
    psi.to_excel(writer, sheet_name='TCL PSI', index=True, merge_cells=False, startrow=1)
    gap_month.to_excel(writer, sheet_name='PP Summary', startrow=1)

In [73]:
wb = load_workbook(f'D:/MNT/TCL/PSI/{filename}')

color = PatternFill(fgColor='9999FF', fill_type='solid') # Blue
big_size = Font(name='맑은 고딕', size=20, bold=True, color='000099')
middle_size = Font(name='맑은 고딕', size=15, bold=True, color='000099')

border_style = Side(border_style="thin", color="000000")
border = Border(top=border_style, right=border_style, bottom=border_style, left=border_style)

In [74]:
sh = wb[wb.sheetnames[0]]
sh['A1'] = f'■ TCL Monitor PSI_updated at : {update_time}'
sh['A1'].font = middle_size
sh['F1'] = 'Sub TTL'
for i in range(7, sh.max_column+1):
    col_letter = get_column_letter(i)
    sh.cell(row=1, column=i).value = f'=subtotal(9, {col_letter}3:{col_letter}{sh.max_row})'
    sh.cell(row=1, column=i).border = border
    sh.cell(row=1, column=i).number_format = '#,##0;[RED]Δ#,##0;"-"'
for cell in sh[2]:
    cell.fill = color
    cell.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)
for row in sh.iter_rows(min_row=3, min_col=7, max_row=sh.max_row, max_col=sh.max_column):
    for cell in row:
        cell.border = border
        cell.number_format = '#,##0;[RED]Δ#,##0;"-"'
sh.freeze_panes = 'G3'
sh.column_dimensions['B'].width = 10
sh.column_dimensions['D'].width = 17.5
sh.column_dimensions['E'].width = 23
sh.column_dimensions['F'].width = 10

sh = wb[wb.sheetnames[1]]
sh['A1'] = f'■ TCL PP Gap from previous version'
sh['A1'].font = middle_size
sh['G1'] = 'Sub TTL'
for i in range(8, sh.max_column+1):
    col_letter = get_column_letter(i)
    sh.cell(row=1, column=i).value = f'=subtotal(9, {col_letter}7:{col_letter}{sh.max_row})'
    sh.cell(row=1, column=i).border = border
    sh.cell(row=1, column=i).number_format = '#,##0;[RED]Δ#,##0;"-"'
for row in sh.iter_rows(min_row=2, min_col=len(gap_month.index[0]) ,max_row=2+len(gap_month.columns[0])-1, max_col=sh.max_column):
    for cell in row:
        cell.fill = color
        cell.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)
merge_range = f'{get_column_letter(sh.max_column)}{2}:{get_column_letter(sh.max_column)}{2+len(gap_month.columns[0])-1}'
sh.merge_cells(merge_range)
for cell in sh[merge_range][0]:
    cell.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)
for row in sh.iter_rows(min_row=6, min_col=1 ,max_row=6, max_col=len(gap_month.index[0])):
    for cell in row:
        cell.fill = color
        cell.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)
for row in sh.iter_rows(min_row=7, min_col=8, max_row=sh.max_row, max_col=sh.max_column):
    for cell in row:
        cell.border = border
        cell.number_format = '#,##0;[RED]Δ#,##0;"-"'
sh.freeze_panes = 'H7'
sh.column_dimensions['B'].width = 10
sh.column_dimensions['D'].width = 17.5
sh.column_dimensions['E'].width = 23
sh.column_dimensions['G'].width = 10.5

for name in wb.sheetnames:
    wb[name].sheet_view.showGridLines = False
wb.save(f'D:/MNT/TCL/PSI/{filename}')
wb.close()

In [382]:
psi.columns.name = 'Week Name'

In [386]:
psi = psi.stack('Week Name').reset_index()
psi = psi.rename(columns={0:'QTY'})

In [388]:
psi['Month'] = psi['Week Name'].apply(lambda x:f'{datetime.date.fromisoformat(x[:10]).isocalendar().year}-{get_month_from_date(datetime.date.fromisoformat(x[:10])):02d}')

In [389]:
psi

,Region,Country,To Site,Mapping Model.Suffix,Category,Week Name,QTY,Month
0,SEA,Philippines,EACM,22MR410-B.APHQ,1_BOH,2023-08-07(W32),0,2023-08
1,SEA,Philippines,EACM,22MR410-B.APHQ,1_BOH,2023-08-14(W33),0,2023-08
2,SEA,Philippines,EACM,22MR410-B.APHQ,1_BOH,2023-08-21(W34),0,2023-08
3,SEA,Philippines,EACM,22MR410-B.APHQ,1_BOH,2023-08-28(W35),0,2023-08
4,SEA,Philippines,EACM,22MR410-B.APHQ,1_BOH,2023-09-04(W36),0,2023-09
...,...,...,...,...,...,...,...,...
13567,SCA,Chile,ESCL_STG,27MR400-B.AWHQ,6_SP Gap,2024-01-01(W01),-2048,2024-01
13568,SCA,Chile,ESCL_STG,27MR400-B.AWHQ,6_SP Gap,2024-01-08(W02),-2048,2024-01
13569,SCA,Chile,ESCL_STG,27MR400-B.AWHQ,6_SP Gap,2024-01-15(W03),-2048,2024-01
13570,SCA,Chile,ESCL_STG,27MR400-B.AWHQ,6_SP Gap,2024-01-22(W04),-2048,2024-01


In [205]:
sp = sp.pivot_table('QTY', index=['Region', 'Country', 'To Site', 'Mapping Model.Suffix'], columns='Week Name', aggfunc=sum).fillna(0).astype(int)

In [216]:
pd.concat([sp, pp]).pivot_table('QTY', index=['Region', 'Country', 'To Site', 'Mapping Model.Suffix', 'Category'], columns='Week Name', aggfunc=sum)

Week Name                                               2023-08-07(W32)  \
Region Country  To Site  Mapping Model.Suffix Category                    
EU     Czech    EEPL_CZ  22MR410-B.AEUQ       SP                    NaN   
                         27MR400-B.AEUQ       SP                    NaN   
       France   EEFS     22MR410-B.AEUQ       SP                    NaN   
                         27MR400-B.AEUQ       SP                    NaN   
       Hungary  EEMK     22MR410-B.AEUQ       PP                  840.0   
...                                                                 ...   
SEA    Thailand EATH     22MR410-B.ATMQ       SP                    NaN   
                         27MR400-B.ATMQ       SP                    NaN   
       Vietnam  EAVH_HCM 22MR410-B.ATVQ       SP                    NaN   
                         27MR400-B.ATVQ       SP                    NaN   
                EAVH_HN  27MR400-B.ATVQ       SP                    NaN   

Week Name                                               2023-08-14(W33)  \
Region Country  To Site  Mapping Model.Suffix Category                    
EU     Czech    EEPL_CZ  22MR410-B.AEUQ       SP                    NaN   
                         27MR400-B.AEUQ       SP                    NaN   
       France   EEFS     22MR410-B.AEUQ       SP                    NaN   
                         27MR400-B.AEUQ       SP                    NaN   
       Hungary  EEMK     22MR410-B.AEUQ       PP                    NaN   
...                                                                 ...   
SEA    Thailand EATH     22MR410-B.ATMQ       SP                 2184.0   
                         27MR400-B.ATMQ       SP                    NaN   
       Vietnam  EAVH_HCM 22MR410-B.ATVQ       SP                    NaN   
                         27MR400-B.ATVQ       SP                    NaN   
                EAVH_HN  27MR400-B.ATVQ       SP                    NaN   

Week Name                                               2023-08-21(W34)  \
Region Country  To Site  Mapping Model.Suffix Category                    
EU     Czech    EEPL_CZ  22MR410-B.AEUQ       SP                    NaN   
                         27MR400-B.AEUQ       SP                    NaN   
       France   EEFS     22MR410-B.AEUQ       SP                    NaN   
                         27MR400-B.AEUQ       SP                    NaN   
       Hungary  EEMK     22MR410-B.AEUQ       PP                    NaN   
...                                                                 ...   
SEA    Thailand EATH     22MR410-B.ATMQ       SP                 2184.0   
                         27MR400-B.ATMQ       SP                    NaN   
       Vietnam  EAVH_HCM 22MR410-B.ATVQ       SP                    NaN   
                         27MR400-B.ATVQ       SP                    NaN   
                EAVH_HN  27MR400-B.ATVQ       SP                    NaN   

Week Name                                               2023-08-28(W35)  \
Region Country  To Site  Mapping Model.Suffix Category                    
EU     Czech    EEPL_CZ  22MR410-B.AEUQ       SP                    NaN   
                         27MR400-B.AEUQ       SP                    NaN   
       France   EEFS     22MR410-B.AEUQ       SP                    NaN   
                         27MR400-B.AEUQ       SP                    NaN   
       Hungary  EEMK     22MR410-B.AEUQ       PP                    NaN   
...                                                                 ...   
SEA    Thailand EATH     22MR410-B.ATMQ       SP                    NaN   
                         27MR400-B.ATMQ       SP                    NaN   
       Vietnam  EAVH_HCM 22MR410-B.ATVQ       SP                    NaN   
                         27MR400-B.ATVQ       SP                    NaN   
                EAVH_HN  27MR400-B.ATVQ       SP                    NaN   

Week Name                                               2023-09-04(W36)  \
Region Country  To Si

In [133]:
pp

,Mapping Model.Suffix,To Site,QTY,Week Name,Region,Country,Category
0,22MR410-B.AEUQ,EEES,840,2023-08-21(W34),EU,Spain,PP
1,22MR410-B.AEUQ,EEMK,840,2023-08-07(W32),EU,Hungary,PP
2,22MR410-B.APDQ,EMTK,840,2023-08-14(W33),MEA,Turkey,PP
3,22MR410-B.APDQ,EMTK,840,2023-08-28(W35),MEA,Turkey,PP
4,22MR410-B.ATIQ,EAIN_DC,2184,2023-08-28(W35),SEA,Indonesia,PP
5,22MR410-B.ATMQ,EATH,2184,2023-08-14(W33),SEA,Thailand,PP
6,22MR410-B.ATMQ,EATH,2184,2023-08-21(W34),SEA,Thailand,PP
7,22MR410-B.AWPQ,ESCB,840,2023-08-28(W35),SCA,Columbia,PP


In [134]:
sp

,To Site,Region,Country,Mapping Model.Suffix,Week Name,Category,QTY
0,EACM,SEA,Philippines,22MR410-B.APHQ,2023-10-09(W41),SP,840
1,EACM,SEA,Philippines,22MR410-B.APHQ,2023-11-27(W48),SP,840
2,EACM,SEA,Philippines,22MR410-B.APHQ,2024-01-01(W01),SP,840
3,EACM,SEA,Philippines,24MR400-B.APHQ,2023-10-02(W40),SP,3060
4,EACM,SEA,Philippines,24MR400-B.APHQ,2023-10-30(W44),SP,1530
...,...,...,...,...,...,...,...
263,ESCB,SCA,Columbia,24MR400-B.AWPQ,2023-12-04(W49),SP,800
264,ESCL_STG,SCA,Chile,22MR410-B.AWHQ,2023-09-18(W38),SP,1680
265,ESCL_STG,SCA,Chile,22MR410-B.AWHQ,2023-10-16(W42),SP,840
266,ESCL_STG,SCA,Chile,27MR400-B.AWHQ,2023-09-18(W38),SP,1024


In [136]:
psi = pd.concat([sp, pp])

In [137]:
psi.head(3)

,To Site,Region,Country,Mapping Model.Suffix,Week Name,Category,QTY
0,EACM,SEA,Philippines,22MR410-B.APHQ,2023-10-09(W41),SP,840
1,EACM,SEA,Philippines,22MR410-B.APHQ,2023-11-27(W48),SP,840
2,EACM,SEA,Philippines,22MR410-B.APHQ,2024-01-01(W01),SP,840


In [140]:
psi.pivot_table('QTY', index=['Region', 'Country', 'To Site', 'Mapping Model.Suffix', 'Category'], columns='Week Name', aggfunc=sum).fillna(0).astype(int).to_clipboard()

In [98]:
pd.concat([sp[['To Site', 'Region', 'Country', 'Mapping Model.Suffix', 'Week Name', 'SP']], pp])

,To Site,Region,Country,Mapping Model.Suffix,Week Name,SP,PP
0,EACM,SEA,Philippines,22MR410-B.APHQ,2023-10-09(W41),840.0,NaN
1,EACM,SEA,Philippines,22MR410-B.APHQ,2023-11-27(W48),840.0,NaN
2,EACM,SEA,Philippines,22MR410-B.APHQ,2024-01-01(W01),840.0,NaN
3,EACM,SEA,Philippines,24MR400-B.APHQ,2023-10-02(W40),3060.0,NaN
4,EACM,SEA,Philippines,24MR400-B.APHQ,2023-10-30(W44),1530.0,NaN
...,...,...,...,...,...,...,...
3,EMTK,MEA,Turkey,22MR410-B.APDQ,2023-08-28(W35),NaN,840.0
4,EAIN_DC,SEA,Indonesia,22MR410-B.ATIQ,2023-08-28(W35),NaN,2184.0
5,EATH,SEA,Thailand,22MR410-B.ATMQ,2023-08-14(W33),NaN,2184.0
6,EATH,SEA,Thailand,22MR410-B.ATMQ,2023-08-21(W34),NaN,2184.0


In [21]:
pic[pic['Subsidiary'].str.contains('SJ')]

,Team,SCM,Position,Subsidiary,Email
40,SCM,오석현,책임,LGESJ,seokhyun.oh@lge.com
89,MKT,변영탁,사원,LGESJ,youngtak.byun@lge.com


In [5]:
pic[pic['Subsidiary'].str.contains('DG')]

,Team,SCM,Position,Subsidiary,Email
7,SCM,송명규,책임,LGEDG,danny.song@lge.com
61,MKT,윤석규,책임,LGEDG,sk.yoon@lge.com


In [92]:
sp = sp[sp['From Site'].str.contains('TCL')]
sp = sp[sp['Frozen'] == 'Y']
sp = pd.merge(sp, site[['To Site', 'Subsidiary', 'Country']], how='left').drop(columns=['Planweek', 'Created_at', 'Division', 'Rep PMS', 'Category', 'Frozen'])

In [93]:
pic['Subsidiary'] = pic['Subsidiary'].apply(lambda x:x.split(' ')[0])

In [94]:
po = po.reset_index()
po['Week Name'] = po['RSD'].apply(get_weekname)
po = po[po['Cancel'] == 'N']

In [95]:
sp = pd.merge(sp, po.drop(columns=['Cancel', 'Issued Date', 'RSD', 'Method', 'Price Term', 'Currency', 'Payment Term']), how='left')

In [97]:
sp.to_clipboard()

In [58]:
pic[pic['Team'] == 'MKT']

,Team,SCM,Position,Subsidiary,Email
55,MKT,장진현,선임,LGECI,jinhyeon.jang@lge.com
56,MKT,민병우,책임,LGEUS,joseph.min@lge.com
57,MKT,박준성,책임,LGEUS,joonsung23.park@lge.com
58,MKT,김재영,사원,LGEJP,jaeyoung3.kim@lge.com
59,MKT,오세림,사원,LGEBN,selim1.oh@lge.com
60,MKT,김한주,책임,LGECZ,hanju.kim@lge.com
61,MKT,윤석규,책임,LGEDG,sk.yoon@lge.com
62,MKT,김원중,책임,LGEES,wj.kim@lge.com
63,MKT,김지환,책임,LGEFS,jacky5.kim@lge.com
64,MKT,이택준,선임,LGEHS,taekjun.lee@lge.com
